In [661]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import scipy
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import tensorflow as tf
import pandas as pd
from surprise import SVD, Reader, Dataset, SVDpp
from surprise.model_selection import train_test_split, GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import implicit
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

In [336]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d
allHours = []
allgames = set()
itemPerUser_all = defaultdict(set)
userPerItem_all = defaultdict(set)
userIDs = {}
itemIDs = {}
all_data = []
for l in readJSON("train.json.gz"):
    u,i = l[0],l[1]
    allHours.append(l)
    allgames.add(l[1])
    all_data.append(l[2])
    itemPerUser_all[l[0]].add(l[1])
    userPerItem_all[l[1]].add(l[0])
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
nUsers,nItems = len(userIDs),len(itemIDs)


In [636]:
hoursTrain = allHours[:155000]
hoursTrain2 = allHours[155000:175000]
hoursValid = allHours[155000:]
user_hours = defaultdict(list)
item_hours = defaultdict(list)
itemPerUser = defaultdict(set)
userPerItem = defaultdict(set)
gameCount = defaultdict(int)
totalPlayed = 0
interactions = []
for d in allHours:
    u,i = d[0],d[1]
    user_hours[u].append(d[2])
    item_hours[i].append(d[2])
    gameCount[i] += 1
    totalPlayed += 1
for d in hoursTrain: 
    u,i = d[0],d[1]
    itemPerUser[u].add(i)
    userPerItem[i].add(u)
    interactions.append((u,i,1))
for u in user_hours:
    user_hours[u].sort(key=lambda x: x['date'])
    
for i in item_hours:
    item_hours[i].sort(key=lambda x: x['date'])

In [637]:
##################################################
# Play prediction                                #
##################################################

In [638]:
items = list(itemIDs.keys())
allgames = list(allgames)
def randomGame():
    return allgames[random.randint(0, len(allgames)) - 1]
simlulated_vali = {}
for d in hoursValid:
    simlulated_vali[(d[0], d[1])] = 1
    game = randomGame()
    while(game in itemPerUser_all[d[0]] or (d[0], game) in simlulated_vali.keys()):
        game = randomGame()
    simlulated_vali[(d[0], game)] = 0

simlulated_train2 = {}
for d in hoursTrain2:
    simlulated_train2[(d[0], d[1])] = 1
    game = randomGame()
    while(game in itemPerUser_all[d[0]] or (d[0], game) in simlulated_train2.keys()):
        game = randomGame()
    simlulated_train2[(d[0], game)] = 0
len(simlulated_train2)

39998

In [639]:
def key_to_IUlist(keys):
    I = []
    U = []
    for key in keys:
        if (key[0] not in userIDs.keys()):
            U.append(0)
            print('!!')
        else:
            U.append(userIDs[key[0]])
        if (key[1] not in itemIDs.keys()):
            I.append(0)
            print('!!')
        else:
            I.append(itemIDs[key[1]])
        
    return U,I
testU,testI = key_to_IUlist(simlulated_vali.keys())

In [640]:
class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb, lamb2):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb
        self.lamb2 = lamb2

    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI))/10 +\
               self.lamb2* (tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))/10
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui

    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))

In [641]:
def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [], [], []
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(items) # negative sample
            while j in itemPerUser_all[u]:
                j = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleJ.append(itemIDs[j])

        loss = model(sampleU,sampleI,sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [742]:
optimizer = tf.keras.optimizers.Adam(0.03)
modelBPR = BPRbatch(10, 0.00013, 0.00018)
for i in range(300):
    obj = trainingStepBPR(modelBPR, interactions)
    if (i % 10 == 9):
        print("iteration " + str(i+1) + ", objective = " + str(obj))
        result = modelBPR.score(testU, testI)
        pred_played = {x:(lambda x: 1 if x > 0.5 else 0)(y) for x,y in zip(simlulated_vali.keys(), result)}
        accuracy = sum([pred_played[x] == simlulated_vali[x] for x in pred_played.keys()])/len(pred_played)
        print(i, accuracy)

iteration 10, objective = 0.6087601
9 0.5
iteration 20, objective = 0.52873313
19 0.6858842942147108
iteration 30, objective = 0.511766
29 0.6946597329866493
iteration 40, objective = 0.49955523
39 0.6963598179908995
iteration 50, objective = 0.4935807
49 0.6984849242462123
iteration 60, objective = 0.48529798
59 0.7042102105105256
iteration 70, objective = 0.47861168
69 0.7098104905245263
iteration 80, objective = 0.4750282
79 0.7122606130306516
iteration 90, objective = 0.4709122
89 0.7124606230311515
iteration 100, objective = 0.46832374
99 0.715810790539527
iteration 110, objective = 0.46748367
109 0.7179108955447773
iteration 120, objective = 0.46423542
119 0.7178608930446523
iteration 130, objective = 0.46218514
129 0.7197609880494025
iteration 140, objective = 0.4615872
139 0.7196859842992149
iteration 150, objective = 0.46054292
149 0.719885994299715
iteration 160, objective = 0.46174118
159 0.7187359367968399
iteration 170, objective = 0.46190444
169 0.7197109855492775
iterati

In [643]:
result_np = result.numpy()
pred_played = {x:y for x,y in zip(simlulated_vali.keys(), result_np)}

In [644]:
pred_played = {x:(lambda x: 1 if x > 0.5 else 0)(y) for x,y in zip(simlulated_vali.keys(), result)}
accuracy = sum([pred_played[x] == simlulated_vali[x] for x in pred_played.keys()])/len(pred_played)
print(i, accuracy)

299 0.7273113655682785


In [645]:
def cosine(s1, s2):
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [646]:
def cosine_simi(input_pairs):
    result_played = []
    for pair in input_pairs:
        u = pair[0]
        i = pair[1]
        items = itemPerUser[u]
        similarities_i = []
        for item in items:
            if item != i:
                similarities_i.append(cosine(userPerItem[item], userPerItem[i]))
        users = userPerItem[i]
        similarities_u = []
        for user in users:
            if user != u:
                similarities_u.append(cosine(itemPerUser[user], itemPerUser[u]))
        similarities_i.sort(reverse = True)
        similarities_u.sort(reverse = True)
        result = []
        if (len(similarities_i) == 0):
            result.append(0.01)
            result.append(0.01)
        else:
            result.append(sum(similarities_i)/len(similarities_i))
            result.append(similarities_i[0])
        if (len(similarities_u) == 0):
            result.append(0.01)
            result.append(0.01)
        else:
            result.append(sum(similarities_u)/len(similarities_u))
            result.append(similarities_u[0])
        result_played.append(result)
    return result_played

In [701]:
def pipline_wp(input_pairs):
    simis = cosine_simi(input_pairs)
    bpr_result = modelBPR.score(*key_to_IUlist(input_pairs)).numpy()
    times_played = [gameCount[x[1]]/100 for x in input_pairs]
    X = [[x, *y,z, 1] for x,y,z in zip(bpr_result, simis, times_played)]
    return X

In [743]:
X = pipline_wp(simlulated_train2)
Y = list(simlulated_train2.values())

In [744]:
model = linear_model.LogisticRegression(fit_intercept=False)
model.fit(X,Y)

LogisticRegression(fit_intercept=False)

In [745]:
x_pred = pipline_wp(simlulated_vali)
y_pred = model.predict(x_pred)

In [746]:
result = [x==y for x,y in zip(y_pred, simlulated_vali.values())]
sum(result)/len(result)

0.7278113905695285

In [747]:
pred_x = []
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        continue
    u,i = l.strip().split(',')
    pred_x.append((u,i))
x = pipline_wp(pred_x)

!!
!!


In [748]:
pred_val = model.decision_function(x)
pred_val

array([ 1.20809417, -0.4204679 , -0.7810907 , ...,  1.52428368,
       -0.43916725,  1.66613393])

In [749]:
def half_positive(keys, pred_values):
    pred_map = {x:y for x,y in zip(keys, pred_values)}
    result = {}
    itemPerUser_test = defaultdict(list)
    for pair in keys:
        u,i = pair[0], pair[1]
        itemPerUser_test[u].append((i, pred_map[(u,i)]))
    for u in itemPerUser_test:
        pred_u = itemPerUser_test[u]
        pred_u = sorted(pred_u, key = lambda x: x[1], reverse = True)
        thres = pred_u[len(pred_u)//2][1]
        for ui in pred_u:
            if (ui[1] > thres):
                result[(u,ui[0])] = 1
            else:
                result[(u,ui[0])] = 0
    return result
final_pred = half_positive(pred_x, pred_val)

In [752]:
predictions = open("predictions_Played.csv", 'w')
predictions.write('userID,gameID,prediction\n')
for x_ui,pred in zip(pred_x, final_pred):
    predictions.write(x_ui[0] + ',' + x_ui[1] + ","+str(final_pred[(x_ui[0], x_ui[1])])+"\n")
predictions.close()

In [753]:
trainHours = [r[2]['hours_transformed'] for r in allHours]
trainData = [(r[0], r[1], r[2]['hours_transformed']) for r in hoursTrain]
valiData = [(r[0], r[1], r[2]['hours_transformed']) for r in hoursValid]
trainData_all = [(r[0], r[1], r[2]['hours_transformed']) for r in allHours]
mu = [sum(trainHours) * 1.0 / len(trainHours)]
vali_input_i = [itemIDs[x[1]] for x in hoursValid]
vali_input_u = [userIDs[x[0]] for x in hoursValid]
vali_y = np.array([x[2]['hours_transformed'] for x in hoursValid])

In [ ]:
df = pd.DataFrame(trainData, columns=['userID', 'itemID', 'rating'])
reader = Reader(rating_scale=(min(trainHours), max(trainHours)))
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

In [ ]:
param_grid = {
    'n_factors': [150],
    'lr_all': [0.003],
    'reg_bu': [0.4],
    'reg_bi': [0.003],
    'reg_pu': [0.47],
    'reg_qi': [0.4],
    'reg_yj': [0.02],
    'n_epochs': [75]
}
gs = GridSearchCV(SVDpp, param_grid, measures=['mse'], cv=36, joblib_verbose = 10, n_jobs=-1)
gs.fit(data)
# print(gs.best_score['mse'])
# print(gs.best_params['mse'])

In [ ]:
df = pd.DataFrame(trainData_all, columns=['userID', 'itemID', 'rating'])
reader = Reader(rating_scale=(min(trainHours), max(trainHours)))
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
df_vali = pd.DataFrame(valiData, columns=['userID', 'itemID', 'rating'])
data_vali = Dataset.load_from_df(df_vali[['userID', 'itemID', 'rating']], reader)
trainset = data.build_full_trainset()
model = SVDpp(n_factors= 150, lr_all=0.003, reg_bu=0.4, reg_bi= 0.003, reg_pu= 0.47, reg_qi= 0.4,reg_yj=0.02, n_epochs= 75)
model.fit(trainset)

In [ ]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,i = l.strip().split(',')
    pred = str(model.predict(u, i).est)
    predictions.write(u + ',' + i + ',' + pred + '\n')

predictions.close()